In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import torch
import logging
from transformers import AutoModelForCausalLM
from nnsight import LanguageModel

sys.path.append("../")
import src.utils.logging_utils as logging_utils
import src.functional as functional
import src.models as models
import src.tokens as tokens
import src.dataset as dataset
from src.models import ModelandTokenizer
import src.patchscope_eval as patchscope_eval
import proto.patchscope_pb2 as patchscope_pb2
import src.dataset_manager as dataset_manager

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

2024-11-01 17:59:03 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
# MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.2-3B"
MODEL_KEY = "meta-llama/Llama-3.1-8B"

# MODEL_KEY = "google/gemma-2-9b-it"
# MODEL_KEY = "google/gemma-2-2b"

mt = models.ModelandTokenizer(
    model_key=MODEL_KEY,
    torch_dtype=torch.bfloat16,
)

mt.n_layer

2024-11-01 17:59:03 src.models WARNING  meta-llama/Llama-3.1-8B not found in models
If not found in cache, model will be downloaded from HuggingFace to cache directory


2024-11-01 17:59:05 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:

base_prompt = '''{} Please answer yes or no: '''

instruct_prompt = '''<|start_header_id|>user<|end_header_id|>

{} Please answer yes or no.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

'''

prompt_format = {
    "meta-llama/Llama-3.2-3B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.1-8B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.2-3B" : base_prompt,
    "meta-llama/Llama-3.1-8B" : base_prompt,
}[MODEL_KEY]

model_short_name = {
    "meta-llama/Llama-3.2-3B-Instruct" : "llama_3b_instruct",
    "meta-llama/Llama-3.1-8B-Instruct" : "llama_8b_instruct",
    "meta-llama/Llama-3.2-3B" : "llama_3b",
    "meta-llama/Llama-3.1-8B" : "llama_8b",
}[MODEL_KEY]

In [ ]:
done = []
evaluation_results = []
for dataset_group in dataset_manager.DatasetManager.list_datasets_by_group():
    if dataset_group in done:
        continue
    dataset = dataset_manager.DatasetManager.from_dataset_group(dataset_group, batch_size=16)
    evaluation_config = patchscope_pb2.EvaluationConfig(
        model_key=MODEL_KEY,
        dataset=dataset_group,
        patchscope_config=patchscope_pb2.PatchscopeConfig(
            source_layers=list(range(mt.n_layer)),
            target_layers=[]
        ),
        interested_tokens=["Yes", "No"],
        prompt_format=prompt_format,
    )
    evaluation_runner = patchscope_eval.EvaluationRunner(evaluation_config, mt)
    evaluation_result = evaluation_runner.evaluate(dataset, max_examples=5000)
    print(dataset_group, evaluation_result.accuracy)
    evaluation_results.append((dataset_group, evaluation_result))
    functional.free_gpu_cache()

  0%|                                                                                         | 0/1375 [00:00<?, ?it/s]You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not load library libnvrtc.so.12. Error: libnvrtc.so.12: cannot open shared object file: No such file or directory
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Could not load library libnvrtc.so.12. Error: libnvrtc.so.12: cannot open shared object file: No such file or directory
 23%|█████████████████▉                                                             | 312/1375 [13:56<47:30,  2.68s/it]


language_identification 0.5063897763578274


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:59<00:00,  2.58s/it]


singular_plural 0.5054347826086957
